In [87]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [88]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [90]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model, xception.preprocess_input)

In [91]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [92]:
# bf_val_x = model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

In [104]:
# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [94]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [161]:
preds = []
nb_runs = 3
for i in range(nb_runs):
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(bf_x, y, epochs=15, batch_size=nb_batches, validation_data=(bf_val_x, y_val))
    
    pred = lm.predict(bf_x_test, batch_size=batch_size, verbose=1)
    preds.append(pred)

Found 8222 images belonging to 120 classes.
514/514 [==============================] - 126s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 1s - loss: 3.1421 - acc: 0.6269 - val_loss: 1.9636 - val_acc: 0.8420
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.4625 - acc: 0.8545 - val_loss: 1.0291 - val_acc: 0.8845
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.8411 - acc: 0.8819 - val_loss: 0.6643 - val_acc: 0.8935
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5861 - acc: 0.8939 - val_loss: 0.5092 - val_acc: 0.9030
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.4603 - acc: 0.8994 - val_loss: 0.4301 - val_acc: 0.9045
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3864 - acc: 0.9085 - val_loss: 0.3842 - val_acc: 0.9050
Epoch 7/15
8222/8222 [==============================] - 0s - loss: 0.3375 - acc: 0.9158 - val_loss: 0.3605 

In [163]:
pred_ensemble = np.stack(preds).mean(axis=0)

In [164]:
pred_ensemble.shape

(10357, 120)

### predict

In [165]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [166]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [167]:
description = 'ensemble_3_data_aug'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [168]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [169]:
!kg submit $submission_file_name -m $description

0.31200
